In [1]:
# 패키지 임포트
import numpy as np
import pandas as pd

In [2]:
import warnings
# 경고메세지 끄기
warnings.filterwarnings(action='ignore')

In [3]:
# seaborn 패키지의 titanic 데이터셋을 load 하시오
import seaborn as sns
titanic = sns.load_dataset("titanic")
titanic.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


In [37]:
titanic.count()

survived       891
pclass         891
sex            891
age            714
sibsp          891
parch          891
fare           891
embarked       889
class          891
who            891
adult_male     891
deck           203
embark_town    889
alive          891
alone          891
dtype: int64

#### 연습문제 1
- 타이타닉 호 승객에 대해서, 성별(sex) 인원수, 나이별(age) 인원수, 선실별(class) 인원수, 사망/생존(alive)인원수를 구하시오.

- 성별 인원수는 인덱스 기준으로 정렬하고
- 나이별 인원수는 값 기준 정렬
- 그 나머지는 임의 기준 정렬 하시오.

In [6]:
# value_counts() : 범주형 데이터의 그룹별 개수 반환
# 성별 인원수는 인덱스 기준으로 정렬하고 : sort_index()
titanic['sex'].value_counts().sort_index()

female    314
male      577
Name: sex, dtype: int64

In [9]:
# 나이별 인원수 값 기준 정렬
titanic['age'].value_counts().sort_values()

74.00     1
34.50     1
0.42      1
0.67      1
66.00     1
         ..
28.00    25
19.00    25
18.00    26
22.00    27
24.00    30
Name: age, Length: 88, dtype: int64

In [15]:
titanic['class'].value_counts().sort_index(ascending=False) # 인덱스 내림차순 정렬 
# titanic.class.value_counts().sort_index(ascending=False)
# class 라는 컬럼명은 예약어이므로 []는 사용 가능, . 연산자 접근은 불가능.

Third     491
Second    184
First     216
Name: class, dtype: int64

In [13]:
# 사망생존 인원수
titanic.alive.value_counts().sort_values()

yes    342
no     549
Name: alive, dtype: int64

In [16]:
cols = 'sex','age','class','alive'
v_count = {}

for col in cols :
    v_count[col] = titanic[col].value_counts()

v_count['sex']

male      577
female    314
Name: sex, dtype: int64

#### 연습문제 2
- 성별에 따른 생존자수와 사망자수의 정보를 갖고 있는 데이터프레임 titanic_y_n 를 생성하시오
    - 0은 생존 1은 사망

In [18]:
# 사망자/생존자 데이터 분리
no = titanic[titanic.alive == 'no']
yes = titanic[titanic.alive == 'yes']

In [20]:
no # 사망자데이터
yes # 생존자데이터

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,1,3,female,15.0,0,0,7.2250,C,Third,child,False,NaN,Cherbourg,yes,True
879,1,1,female,56.0,0,1,83.1583,C,First,woman,False,C,Cherbourg,yes,False
880,1,2,female,25.0,0,1,26.0000,S,Second,woman,False,NaN,Southampton,yes,False
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True


In [25]:
no_s = no.sex.value_counts()
yes_s = yes.sex.value_counts()

no_s
yes_s

female    233
male      109
Name: sex, dtype: int64

In [26]:
pd.DataFrame([no_s,yes_s],index=[0,1])

,male,female
0,468,81
1,109,233


In [27]:
titanic_y_n = pd.DataFrame(titanic.loc[:,['sex','alive']].value_counts(), columns=['빈도수'])

In [28]:
titanic_y_n

,,빈도수
sex,alive,
male,no,468
female,yes,233
male,yes,109
female,no,81


In [29]:
pd.pivot_table(titanic_y_n, index=['alive'], columns=['sex'])

빈도수     
sex   female male
alive            
no        81  468
yes      233  109

#### 연습문제
- 타이타닉호 승객을 사망자와 생존자 그룹으로 나누고(alive)
- 각 그룹에 대해 미성년자, 청년, 중년, 장년, 노년 승객의 비율을 구하시오.
    - bins=[1,15,25,35,60,99]
    - labels=['미성년자','청년','중년','장년','노년']
    - 각 그룹별 비유의 전체 합은 1이 되어야 한다.

In [30]:
# 사망자/생존자 데이터 분리
no=titanic[titanic.alive == 'no']
yes = titanic[titanic.alive == 'yes']

In [32]:
# bins의 범위 유효성 검사
np.min(titanic.age) # <- 최소 값을 1로 시작하면 모든 나이가포함되지 않음
np.max(titanic.age)

80.0

In [33]:
bins=[0,15,25,35,60.99]
# 0~15 구간 : 0살 ,미성년자 <=15
labels = ['미성년자','청년','중년','장년','노년']

In [35]:
# no['age_cut'] = pd.cut(no['age'],bins,labels=labels)

In [36]:
no.head(1)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False


In [ ]:
yes['age_cut'] = pd.cut(yes['age'],bins,labels=labels)

### cut 함수 정리
- 데이터 값을 카테고리 값으로 변환

- 값의 크기를 기준으로하여 카테고리 값으로 변환하고 싶을때
    - cut(data,bins,labels)
        - data : 구간 나눌 실제 값, 
        - bins : 구간 경계값
        - labels: 카테고리값
    - qcut(data,구간수,labels)

In [21]:
#구간을 나눌 실제 값 : 관측 데이터
ages=[0,0.5,4,6,4,5,2,10,21,23,37,15,38,31,61,20,41,31,100]

In [36]:
data = ages

# 구간 경계값
# 구간 최소값 < 구간 <= 구간 최대값
#bins = [-1,4,18,25,35,60,100]
bins = [0,4,18,25,35,60,100]
# 각 구간의 이름 : labels=(카테고리명)
# 순서는 구간(bins)순서와 동일해야 함
labels = ['영유아','미성년자','청년','중년','장년','노년']

In [26]:
cats = pd.cut(data, bins, labels=labels)
cats

['영유아', '영유아', '영유아', '미성년자', '영유아', ..., '노년', '청년', '장년', '중년', '노년']
Length: 19
Categories (6, object): ['영유아' < '미성년자' < '청년' < '중년' < '장년' < '노년']

In [27]:
type(cats) # categorical.Categorical

pandas.core.arrays.categorical.Categorical

In [28]:
# 사용 시 list 변환 후 사용하는 게 좋음
list(cats)

['영유아',
 '영유아',
 '영유아',
 '미성년자',
 '영유아',
 '미성년자',
 '영유아',
 '미성년자',
 '청년',
 '청년',
 '장년',
 '미성년자',
 '장년',
 '중년',
 '노년',
 '청년',
 '장년',
 '중년',
 '노년']

##### Categorical 클래스 객체
- 카테고리명 속성 : Categorical.categories
    - 카테고리명 저장
- 코드 속성 : Categorical.codes 
    - 인코딩한 카테고리 값을 정수로 갖는다.

In [30]:
type(cats)

pandas.core.arrays.categorical.Categorical

In [32]:
cats.categories # cats 변수내의 범주 index를 저장

Index(['영유아', '미성년자', '청년', '중년', '장년', '노년'], dtype='object')

In [33]:
cats

['영유아', '영유아', '영유아', '미성년자', '영유아', ..., '노년', '청년', '장년', '중년', '노년']
Length: 19
Categories (6, object): ['영유아' < '미성년자' < '청년' < '중년' < '장년' < '노년']

In [37]:
cats.codes # 카테고리 변환값에 대하여 범주 index내의 위치값
# code 결과에 -1이 있으면 NaN 값을 코드화 한 내용

array([0, 0, 0, 1, 0, 1, 0, 1, 2, 2, 4, 1, 4, 3, 5, 2, 4, 3, 5],
      dtype=int8)

##### 구간 경계선을 지정하지 않고 데이터 개수가 같도록 지정한 수의 구간으로 분할하기 :  qcut()  

- 형식 : pd.qcut(data,구간수,labels=[d1,d2....])
    
    
    - 예)1000개의 데이터를 4구간으로 나누려고 한다면
        - qcut 명령어를 사용 한 구간마다 250개씩 나누게 된다.
        - 예외)같은 숫자인 경우에는 같은 구간으로 처리한다.

In [41]:
# 랜덤정수 20개를 생성하고 생성된 정수를 4개의 구간으로 나누시오.

# 각 구간의 label은 Q1,Q2,Q3,Q4 로 설정하시오.

np.random.seed(2)
data = np.random.randint(20,size=20)
data

array([ 8, 15, 13,  8, 11, 18, 11,  8,  7,  2, 17, 11, 15,  5,  7,  3,  6,
        4, 10, 11])

In [42]:
qcat = pd.qcut(data, 4, labels=['Q1','Q2','Q3','Q4'])
qcat

['Q2', 'Q4', 'Q4', 'Q2', 'Q3', ..., 'Q1', 'Q1', 'Q1', 'Q3', 'Q3']
Length: 20
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [43]:
np.sort(data)

array([ 2,  3,  4,  5,  6,  7,  7,  8,  8,  8, 10, 11, 11, 11, 11, 13, 15,
       15, 17, 18])

In [44]:
pd.value_counts(qcat)

Q1    5
Q2    5
Q3    5
Q4    5
dtype: int64

In [45]:
df0 = pd.DataFrame(data, columns=['관측수'])
df0['범주']=qcat
df0

,관측수,범주
0,8,Q2
1,15,Q4
2,13,Q4
3,8,Q2
4,11,Q3
5,18,Q4
6,11,Q3
7,8,Q2
8,7,Q2
9,2,Q1


#### 연습문제
- ['id_1','id_2','id_3','id_4'] 와 같은 리스트 요소를
- 내포 for문을 이용해서 생성후 리스트로 추가하시오.

In [ ]:
# 고정문자열(id_)과 반복 요소(1,2,3,4)를  조합하는 코드를 내포  for문으로 만들어보세요